# Część na 3.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bresenham import bresenham
import imageio
import pydicom
from pydicom.data import get_testdata_files
from scipy import ndimage
from scipy.fftpack import fft, ifft, fftfreq

In [ ]:
alfa = 1
n = 150
l = 90

In [ ]:

def emiterPosition(angle, i, n, r, l, offset):
    x = r * np.cos( np.radians(angle) ) + offset[0]
    y = r * np.sin( np.radians(angle) ) + offset[1]
    return (x,y)

In [ ]:

def sensorPosition(angle, i, n, r, l, offset):
    x = r * np.cos( np.radians(angle) + np.pi - np.radians(l)/2 + i * ( np.radians(l) / (n-1) ) ) + offset[0]
    y = r * np.sin( np.radians(angle) + np.pi - np.radians(l)/2 + i * ( np.radians(l) / (n-1) ) ) + offset[1]
    return (x,y)

In [ ]:
"""
Symulacja laseru przechodzącego liniowo przez obraz image z punktu start do punktu end.
Należy policzyć, ile mocy lasera zostanie pochłonięte, skorzystaj z funkcji bresenham
"""
def beam(image, start, end):
    w, h = image.shape
    x = bresenham(int(start[0]), int(start[1]), int(end[0]), int(end[1]))
    y = np.array(list(x))
    y = y[y[:,0] >= 0]
    y = y[y[:,0] < w]
    y = y[y[:,1] >= 0]
    y = y[y[:,1] < h]
    return np.sum(image[y[:,0],y[:,1]])

In [ ]:
def makeGif(gif, filename):
    scale = gif[-1].max()
    gif = list(map(lambda x: (x/scale*255).astype(np.uint8), gif))
    imageio.mimsave(filename+".gif", gif)

In [ ]:
def radon_iwm(img_gray, alfa, n, l, R, gifFlag=True, filter=False):   
    gif = []
    gif_f = []
    result = np.zeros((n, len(np.arange(-90,90,alfa))))
    result_filtered = np.zeros((n, len(np.arange(-90,90,alfa))))
    for i,angle in enumerate(np.arange(-90,90,alfa)[::-1]):
        result[..., i] = np.array([beam(img_gray, 
                                         emiterPosition(angle, sensor, n, R, l, (img_gray.shape[1]/2, img_gray.shape[0]/2)),
                                         sensorPosition(angle, sensor, n, R, l, (img_gray.shape[1]/2, img_gray.shape[0]/2)))
                                   for sensor in range(n)])
        if gifFlag:
            gif.append(result.copy())
    if gifFlag:
        makeGif(gif, "radon")
    if filter:
        sinogram = result.copy()
        f = fftfreq(sinogram.shape[0]).reshape(-1, 1)
        fourier_filter = 2 * np.abs(f)

        projection = fft(sinogram, axis=0) * fourier_filter
        sinogram = np.real(ifft(projection, axis=0))
        return sinogram/sinogram.max()
    return result/result.max()

In [ ]:
def inverseRadon_iwn(radon, shape, alfa, n, l, R, gifFlag=True, filter=False):
    width, height = shape
    gif = []
    result = np.zeros(shape)
    R = max(shape)*2**.5+10
    for i, angle in enumerate(np.arange(-90,90,alfa)[::-1]):
        for sensor in range(n):
            start = emiterPosition(angle, sensor, n, R, l, (shape[1]/2, shape[0]/2))
            end = sensorPosition(angle, sensor, n, R, l, (shape[1]/2, shape[0]/2))

            x = bresenham(int(start[0]), int(start[1]), int(end[0]), int(end[1]))
            y = np.array(list(x))
            y = y[y[:,0] >= 0]
            y = y[y[:,0] < width]
            y = y[y[:,1] >= 0]
            y = y[y[:,1] < height]
            
            
            result[y[:,0], y[:,1]] += radon[sensor][i]
        if gifFlag:
            gif.append(result.copy()/result.max())
    if gifFlag:
        makeGif(gif, "inverse")
    return result/result.max()

In [ ]:
def projekt1(file, alfa, n, l,gifFlag, filter):
    img = plt.imread(file)
    img_gray = img[...,0]
    plt.imshow(img_gray)
    R = (max(img_gray.shape))*2**.5+10 
    radon = radon_iwm(img_gray, alfa, n, l, R, gifFlag, filter)
    plt.imshow(radon, cmap=plt.cm.bone)
    plt.show()
    iradon = inverseRadon_iwn(radon, img_gray.shape, alfa, n, l, R, gifFlag, filter)
    plt.imshow(iradon, cmap=plt.cm.bone)
    plt.show()
    return iradon

In [ ]:

iradon = projekt1("Kropka.jpg", alfa, n, l, gifFlag=False, filter=False)
plt.imshow(iradon, cmap=plt.cm.bone)
plt.show()

In [ ]:
iradon = projekt1("Kropka.jpg", alfa, n, l, gifFlag=False, filter=True)
plt.imshow(iradon, cmap=plt.cm.bone)
plt.show()

## Część na 4.0

In [ ]:
import os
import tempfile
import datetime

import pydicom
from pydicom import dcmread
from pydicom.dataset import Dataset, FileDataset

In [ ]:
def write_dicom(array, filename="iradon.dcm", name="Test^Firstname", patientID="123456"):
    # Create some temporary filenames
    print("Setting file meta information...")
    filename_little_endian = tempfile.NamedTemporaryFile(suffix=".dcm").name
    # Populate required values for file meta information
    file_meta = Dataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
    file_meta.MediaStorageSOPInstanceUID = "1.2.3"
    file_meta.ImplementationClassUID = "1.2.3.4"

    print("Setting dataset values...")
    # Create the FileDataset instance (initially no data elements, but file_meta
    # supplied)
    ds = FileDataset(filename_little_endian, {},
                     file_meta=file_meta, preamble=b"\0" * 128)

    # Add the data elements -- not trying to set all required here. Check DICOM
    # standard
    ds.PatientName = name
    ds.PatientID = patientID

    # Set the transfer syntax
    ds.is_little_endian = True
    ds.is_implicit_VR = True

    # Set creation date/time
    dt = datetime.datetime.now()
    ds.ContentDate = dt.strftime('%Y%m%d')
    timeStr = dt.strftime('%H%M%S.%f')  # long format with micro seconds
    ds.ContentTime = timeStr

    # Set pixel array
    ds.PixelData = np.array(array*255, dtype=np.int8).tobytes() #create photo in bits
    ds.Rows, ds.Columns = array.shape # shape
    ds.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
    
    ds.PixelRepresentation = 0 
    ds.BitsAllocated = 8 # 8bitów na pixels
    ds.SamplesPerPixel = 1 # black photo 
    ds.NumberOfFrames = 1 # it is photo 
    ds.PhotometricInterpretation = "MONOCHROME"
    ds.PlanarConfiguration = 0
    
    print("Writing test file", filename_little_endian)
    ds.save_as(filename)
    print("File saved.")

In [ ]:
write_dicom(iradon)

In [ ]:
def readDicom(filename="iradon.dcm"):
    ds = dcmread(filename)
    # Load dimensions based on the number of rows, columns, and slices (along the Z axis)
    ConstPixelDims = (int(ds.Rows), int(ds.Columns))

    ArrayDicom = np.zeros(ConstPixelDims, dtype=np.int8)
    ArrayDicom = ds.pixel_array 

    plt.imshow(ArrayDicom, cmap=plt.cm.bone)
    plt.show()

    return ArrayDicom

In [ ]:
readDicom()

## Część na 5.0 

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
img = plt.imread("Shepp_logan.jpg")
y_true = img[...,0]
y_true = y_true / y_true.max()

In [ ]:
n = 180
l = 180
s = 180
alfa = 180/s

i. liczba detektorów zmienia się od 90 do 720 z krokiem 90,

In [ ]:
def eksperyment1_1(y_true):
    n = 180 
    l = 180
    s = 180
    alfa = 180/s
    nExperiment = {}
    for n in range(90, 721, 90):
        iradon_mse = list()
        filtered_mse = list()
        iradon = projekt1("Shepp_logan.jpg", alfa, n, l,gifFlag=False, filter=True)
        
        print(n)
        
        plt.imshow(iradon, cmap=plt.cm.bone)
        plt.title(f"Iradon image non filtered. n: {n}, l: {l}, alfa:{alfa}")
        plt.savefig(f"./nExperiment/{n}_nonfiltered.jpg")
        
        nExperiment[n] = mean_squared_error(y_true, iradon, squared=False)
    return nExperiment

In [ ]:
nExperiment = eksperyment1_1(y_true)

ii. liczba skanów zmienia się od 90 do 720 z krokiem 90,

In [ ]:
def eksperyment1_2(y_true):
    n = 180 
    l = 180
    s = 180
    alfa = 180/s
    sExperiment = {}
    for s in range(90, 721, 90):
        iradon_mse = list()
        filtered_mse = list()
        alfa = 180/s
        iradon = projekt1("Shepp_logan.jpg", alfa, n, l,gifFlag=False, filter=False)
        
        print(s)
        
        plt.imshow(iradon, cmap=plt.cm.bone)
        plt.title(f"Iradon image non filtered. n: {n}, l: {l}, alfa:{alfa}")
        plt.savefig(f"./sExperiment/{s}_nonfiltered.jpg")

        sExperiment[s] = mean_squared_error(y_true, iradon, squared=False)
    return sExperiment

In [ ]:
sExperiment = eksperyment1_2(y_true)

In [ ]:
def eksperyment1_3(y_true):
    n = 360 
    l = 270
    s = 180
    alfa = 180/s
    lExperiment = {}
    for l in range(45, 271, 45):
        iradon_mse = list()
        iradon= projekt1("Shepp_logan.jpg", alfa, n, l,gifFlag=False, filter=False)
        print(l)
        
        plt.imshow(iradon, cmap=plt.cm.bone)
        plt.title(f"Iradon image non filtered. n: {n}, l: {l}, alfa:{alfa}")
        plt.savefig(f"./lExperiment/{l}_nonfiltered.jpg")
        
        lExperiment[l] = mean_squared_error(y_true, iradon, squared=False)

    return lExperiment

In [ ]:
lExperiment = eksperyment1_2(y_true)

In [ ]:
def eksperyment2(y_true):
    n = 360 
    l = 270
    s = 360
    alfa = 180/s

        
    iradon= projekt1("Shepp_logan.jpg", alfa, n, l,gifFlag=False, filter=False)
    iradon_filtered= projekt1("Shepp_logan.png", alfa, n, l,gifFlag=False, filter=True)

    rmse = mean_squared_error(y_true, iradon, squared=False)
    rmse_filtered = mean_squared_error(y_true, iradon_filtered, squared=False)

    plt.imshow(iradon, cmap=plt.cm.bone)
    plt.title(f"Iradon image non filtered. RMSE: {rmse}. n: {n}, l: {l}, alfa:{alfa}")
    plt.savefig(f"./4Experiment/nonfiltered.png")
    
    plt.imshow(iradon_filtered, cmap=plt.cm.bone)
    plt.title(f"Iradon image filtered. RMSE: {rmse_filtered}. n: {n}, l: {l}, alfa:{alfa}")
    plt.savefig(f"./4Experiment/filtered.png")

    
    return lExperiment

In [ ]:
eksperyment2(y_true)